# Cargando datos

Uniendo ambos dfs

In [4]:
import pandas as pd

df1 = pd.read_csv('reddit_comments1.csv')
df2 = pd.read_csv('reddit_comments2.csv')

df = pd.concat([df1, df2], ignore_index=True)

df.head()

,Unnamed: 0,Comments,Article_ID
0,0,Labure en un equipo de DS como monito programa...,163wrg7
1,1,![gif](giphy|xT5LMuQroxQi36Hwys|downsized),163wrg7
2,2,"Si, totalmente. En un laburo donde estuve una ...",163wrg7
3,3,Puro humo. El jefe de la parte de datos de la ...,163wrg7
4,4,"Yo trabajo en el área de data science, y hasta...",163wrg7


In [15]:
len(df1)

203

In [16]:
len(df2)

159

In [19]:
len(df)

351

# Limpieza de datos

Nos quedamos solo con aquellos que tienen más de 6 caracteres

In [5]:
df = df[df['Comments'].str.len() > 6]

Eliminamos comentarios duplicados

In [18]:
df = df.drop_duplicates(subset='Comments')

# Verificar que funciona el phrase to embedding

In [6]:
!pip install -q sentence-transformers


[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Probamos generar embeddings

In [7]:


from sentence_transformers import SentenceTransformer

# Modelo multilingüe que soporta español
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

comentarios = [
    "Me gustó mucho el producto",
    "No volvería a comprar aquí"
]

embeddings = model.encode(comentarios)

print(embeddings.shape)  # (2, 384)
print(embeddings[0])     # Vector del primer comentario


c:\Users\quine\Documents\repos_github\getting_comments_from_reddit\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\quine\Documents\repos_github\getting_comments_from_reddit\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\quine\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windo

(2, 384)
[ 1.17575815e-02  7.34429881e-02 -1.83306068e-01 -1.79916620e-01
 -2.88933098e-01 -8.55137035e-03  3.61190647e-01  3.38651627e-01
 -1.50086675e-02 -1.86697338e-02 -4.71796468e-02  1.38983186e-02
  2.49787010e-02  1.84713565e-02 -7.97504485e-02 -1.39127582e-01
  2.96416819e-01 -5.29948249e-02 -1.08694933e-01 -1.96167022e-01
 -3.70307058e-01  2.00011805e-01 -8.85496289e-02  2.54753977e-01
 -1.96138918e-01 -1.01770014e-02 -2.70332638e-02  4.10409011e-02
  2.01787949e-02 -2.63322413e-01  3.99000570e-02  8.93145055e-02
  1.67913690e-01 -6.91020712e-02 -2.20052883e-01  1.04331098e-01
  2.02376135e-02 -3.70090425e-01 -4.46848422e-02 -2.23545298e-01
  6.40539527e-02  1.39296785e-01  8.58055875e-02  1.98045239e-01
  1.55923041e-02  1.60782337e-02  1.51088625e-01 -2.14019671e-01
  4.03685302e-01  1.67928502e-01  1.25912145e-01 -2.86388546e-01
 -2.04541951e-01 -2.18376264e-01  1.73360899e-01  8.29442367e-02
 -5.66976890e-02 -4.85686772e-03 -1.28955543e-01  7.77467191e-02
  6.59535974e-02

Iteramos sobre cada comentarios para guardar el embedding

In [11]:
for idx, row in df.iterrows():
    comentario = row['Comments']
    curr_embedding = model.encode([comentario])[0]
    df.at[idx, 'Embedding'] = curr_embedding.tolist()

    print(f"Processed comment {idx + 1}/{len(df)}: {comentario[:30]}...")  # Print first 30 characters

Processed comment 1/355: Labure en un equipo de DS como...
Processed comment 2/355: ![gif](giphy|xT5LMuQroxQi36Hwy...
Processed comment 3/355: Si, totalmente. En un laburo d...
Processed comment 4/355: Puro humo. El jefe de la parte...
Processed comment 5/355: Yo trabajo en el área de data ...
Processed comment 6/355: En ningún lugar que conozca to...
Processed comment 7/355: Todos los data scientist que c...
Processed comment 8/355: Justito me vi un video interes...
Processed comment 9/355: Analista de datos podría ser, ...
Processed comment 10/355: Yo acompañé a un amigo muy cer...
Processed comment 11/355: Sí, yo estoy haciendo un curso...
Processed comment 12/355: Humo, cuando todos esos egresa...
Processed comment 13/355: Ah la mierda....
Processed comment 14/355: Otra cosa que no entiendo ¿Por...
Processed comment 15/355: Exactamente...
Processed comment 16/355: Mirá la UTN tiene una carrera ...
Processed comment 17/355: Hola, yo quiero estudiar para ...
Processed comment 18/355:

Ya tenemos nuestros embeddings

In [12]:
df.head()

,Unnamed: 0,Comments,Article_ID,Embedding
0,0,Labure en un equipo de DS como monito programa...,163wrg7,"[-0.20420339703559875, 0.039422813802957535, -..."
1,1,![gif](giphy|xT5LMuQroxQi36Hwys|downsized),163wrg7,"[-0.3553767502307892, 0.07863187044858932, 0.1..."
2,2,"Si, totalmente. En un laburo donde estuve una ...",163wrg7,"[0.01886007748544216, 0.20611713826656342, -0...."
3,3,Puro humo. El jefe de la parte de datos de la ...,163wrg7,"[-0.08595969527959824, 0.30381423234939575, -0..."
4,4,"Yo trabajo en el área de data science, y hasta...",163wrg7,"[-0.0449964702129364, -0.08458462357521057, -0..."


# Guardar csv

In [14]:
df.to_csv('reddit_comments_with_embeddings.csv', index=False)